In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Assume look_descriptions is a dictionary {look_id: description}
look_embeddings = {look_id: model.encode(desc) for look_id, desc in look_descriptions.items()}

def recommend_look(user_input):
    user_embedding = model.encode(user_input)
    similarities = {look_id: cosine_similarity([user_embedding], [emb])[0][0] for look_id, emb in look_embeddings.items()}
    best_look_id = max(similarities, key=similarities.get)
    return look_descriptions[best_look_id], merged_df[merged_df["look_id"] == best_look_id]["product_name"].tolist()

# Example usage
user_request = "I want a casual outfit with a blue top"
recommended_description, recommended_products = recommend_look(user_request)
print(f"Recommended: {recommended_description}")
print("Products:", recommended_products)